In [3]:
import nltk
import sqlite3
import os
import re
import sys
from nltk import sent_tokenize

#declare global variables
fixedSentences = []
citations = []
uidarr = []
sentences = []
totcount = 0 
filename = ""
arxiv_id = ""
def readfile(filename):
    
    #open a file
    f = open(filename, 'rt')
    text = f.read()
    f.close()
    return text

def parsefile(text):
    #use string tokenize
    sentences = sent_tokenize(text)
    return sentences

def fixSentences():
    
    #fix sentences. There is a bug in unarXive dataset where two sentences are merged into one. 
    #This happens when a sentence has a Formula, and then the next word starts with a capital letter. 
    
    for s in sentences:
        
        #remove bad formatting
        #remove newlines
        s = re.sub(r"\r|\n", " ", s)
        
        #remove double space
        s = re.sub(r"\s\s+", " ", s)
        
        #remove spacing of punctuation.  Ex: Turning "  ."into ".", "  ," into "," etc. 
        s = re.sub(r" *\.", ".", s)
        s = re.sub(r" *,", ",", s)
        s = re.sub(r" *\).", ")", s)
        
        #prepare second string variable
        s1 = ""

        #find all occurences of "FORMULA", init variable to tell if we divided or not. 
        indices = [m.start() for m in re.finditer("FORMULA", s)]
        divide = False

        #check if the next word after FORMULA has a capital letter in the first one. 
        #this can result in a bug, where it is something like FORMULA CDMP. Will improve soon to check the entire word. 

        
        #list of bugs discovered that can cause noise.
        # REF -> Splits a sentence into 2.
        # FORMULA and FIGURE also can split a sentence into 2.
        # i.e. and e.g. cause a different bug where it splits 1 into 2.
        
        for i in indices:
            if(i+8 < len(s) and s[i+8].isupper() and i+8 not in indices):
                s1 = s[i+8:]
                s = s[:i+8]
                if(s[len(s)-1] == " "):
                    s = s[:len(s)-1]
                s += "."

        #append finish result into array
        if(divide == False): 
            fixedSentences.append(s)
        else: 
            fixedSentences.append(s)
            fixedSentences.append(s1)

def findCitations(conn):
    
    #this is to find the the {{cite:  }}
    
    for i in range(0, len(fixedSentences)):
        s = fixedSentences[i]
        
        #find all indexes of {{cite: and }}. These two arrays store the indexes of these substrings in s. 
        cinarr = [m.start() for m in re.finditer("{{cite:", s)]
        cendarr = [m.start() for m in re.finditer("}}", s)]
        
        for j in range(0, len(cinarr)): #  parse citations
            #get the indexes
            citein = cinarr[j]  
            citeend = cendarr[j]
            
            uid = s[citein+7:citeend]  # get the UID using string splice
            uidarr.append(uid) # append uid into temporaryarray
        
        #replace all of the cites 
        #The Citations will be replaced in order of first occurence, so the first citation will correspond to the first CITE_MARK.
        #Relatively easy to search for all occurences of CITE_MARK and then match them. Probably can write another program to do that
        
        #print(s)
        
        s = re.sub(r"{{cite:.*?}}", "CITE_MARK", s) 
        #fixedSentences[i] = s
        
        citations.append(len(uidarr)) # update amount of citations for this sentence
        
        for uid in uidarr: #append all citations into db
            add_sentence_citation(conn, (arxiv_id, i+totcount, uid))
            
        #add sentence to db
        create_sentence(conn, (arxiv_id, s, citations[i], i+totcount)) #append sentence into db
        
        #clear for next sentence
        uidarr.clear()


def create_sentence(conn, sentence):
    
    #function to insert all sentences into sqlite
    
    sql = ''' INSERT INTO sentences (arxiv_id,sentence,citation_count, sentence_id) 
        VALUES (?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, sentence)
    conn.commit()
    return


def add_sentence_citation(conn, citation):
    
    #function to link all citations to their sentences and insert into sqlite
    
    sql = ''' INSERT INTO citations (arxiv_id, sentence_id,paper_id) 
        VALUES (?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, citation)
    conn.commit()
    return

if(__name__ == "__main__"):
    
    #read input
    directory = "/Users/brian/projects/rnn/thesis-master/unarXive-2020/papers"
    filecounter = 0
    sconn =  sqlite3.connect("sentence.db")
    cconn = sqlite3.connect("citation.db")
    
    for filename in os.listdir(directory):
        #get filename
        fname  = directory+"/"+filename
        arxiv_id = filename[:len(filename)-4]
        #get input
        text = readfile(fname)
        sentences = parsefile(text)

        #fix formatting of the sentences
        fixSentences()

        #find all occurences of {{cite: }}
        findCitations(sconn)

        #add the number of sentences to the total number of sentences
        totcount += len(fixedSentences)
        fixedSentences.clear()
        citations.clear()
        filecounter += 1
        print(filecounter)
        
    sconn.close()
    cconn.close()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204


KeyboardInterrupt: 